In [ ]:
# default_exp models.tripletnet

# Implementaion of a triplet network

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F

A triplet network takes in 3 inputs:
- The query image
- The positive image
- The Negative image
These 3 images all pass through the same embedding extractor

The query image can be a cropped out part of a parent image or an image very similar to the parent image in the embedding space. This parent image will be taken as the postive image because it is very similar to the query image in the embedding space. The negative image however is one that bears no embedding similarity to the positive image.

NB: It is important to note however that there are many notions of similarity between items. Items may be similar in color or in brand or gender use etc. This raises the need for us to have conditions of simialrity while building our architecture

We will be using a conditional similarity network to learn the different notions of similarities from the triplets while training

In [ ]:
#export
class TripletNet(nn.Module):
    def __init__(self, embeddingnet):
        super(TripletNet, self).__init__()
        self.embeddingnet = conditionalnet
        
    def forward(self, q, p, n, c):
        """
        q: Query Image
        p: Positive Image
        n: Negative Image
        c: Condition of similarity
        """
        
        # it is important to normalize the embeddings while using triplets
        embedded_x, masknorm_norm_q, embed_norm_q, tot_embed_norm_q = self.conditionalnet(q, c)
        embedded_p, masknorm_norm_p, embed_norm_p, tot_embed_norm_p = self.conditionalnet(p, c)
        embedded_n, masknorm_norm_n, embed_norm_n, tot_embed_norm_n = self.conditionalnet(n, c)
        mask_norm = (masknorm_norm_q + masknorm_norm_p + masknorm_norm_n) / 3
        embed_norm = (embed_norm_q + embed_norm_p + embed_norm_n) / 3
        mask_embed_norm = (tot_embed_norm_q + tot_embed_norm_p + tot_embed_norm_n) / 3
        
        pos_dist = F.pairwise_distance(embedded_q, embedded_p, 2)
        neg_dist = F.pairwise_distance(embedded_q, embedded_n, 2)
        
        return pos_dist, neg_dist, mask_norm, embed_norm, mask_embed_norm